In [1]:
from ipynb.fs.full import data
from ipynb.fs.full import models

cuda


In [16]:
import os
import gc
import cv2
import math
import numpy as np
import torch
import time
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms
from skimage import transform
from sklearn import model_selection
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from cvzone.HandTrackingModule import HandDetector

In [3]:
#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [14]:
root_dir = 'Mudra Dataset/Single hand'
data = []
if os.path.isdir(root_dir):
    for dirs in os.listdir(root_dir):
        full_path = os.path.join(root_dir, dirs)
        csv_path = os.path.join(full_path, 'Annotation.csv')
        csv_file = pd.read_csv(csv_path)
        for i, row in csv_file.iterrows():
            img_path = os.path.join(full_path, row[0])
            image = cv2.imread(img_path)
            data.append(image)

In [19]:
def compute_joint_angle(data, a, b, c, d):
    jointAngle = []
    dr1 = [data[a][0] - data[0][0], data[a][1] - data[0][1], data[a][2] - data[0][2]]
    dr2 = [data[b][0] - data[a][0], data[b][1] - data[a][1], data[b][2] - data[a][2]]
    dr3 = [data[c][0] - data[b][0], data[c][1] - data[b][1], data[c][2] - data[b][2]]
    dr4 = [data[d][0] - data[c][0], data[d][1] - data[c][1], data[d][2] - data[c][2]]
    
    m1 = math.sqrt(dr1[0] * dr1[0] + dr1[1] * dr1[1] + dr1[2] * dr1[2])
    m2 = math.sqrt(dr2[0] * dr2[0] + dr2[1] * dr2[1] + dr2[2] * dr2[2])
    jointAngle.append(math.degrees(math.acos((dr1[0] * dr2[0] + dr1[1] * dr2[1] + dr1[2] * dr2[2])/(m1*m2))))
    
    m1 = m2
    m2 = math.sqrt(dr3[0] * dr3[0] + dr3[1] * dr3[1] + dr3[2] * dr3[2])
    jointAngle.append(math.degrees(math.acos((dr2[0] * dr3[0] + dr2[1] * dr3[1] + dr2[2] * dr3[2])/(m1*m2))))
    
    m1 = m2
    m2 = math.sqrt(dr4[0] * dr4[0] + dr4[1] * dr4[1] + dr4[2] * dr4[2])
    jointAngle.append(math.degrees(math.acos((dr3[0] * dr4[0] + dr3[1] * dr4[1] + dr3[2] * dr4[2])/(m1*m2))))
    
    return jointAngle

In [10]:
start_time = time.time()

In [20]:
detector = HandDetector(maxHands=2, detectionCon=0.8)
for i in range(len(data)):
    image = data[i]
    hands, img = detector.findHands(image)
    
    result = []
    
    if hands:
        hand = hands[0]
        lmlist = hand['lmList']
        
        index_MCP = lmlist[5]
        little_MCP = lmlist[17]
        centre = [(index_MCP[0]+little_MCP[0])/2, (index_MCP[1]+little_MCP[1])/2, (index_MCP[2]+little_MCP[2])/2]
        for lm in lmlist:
            result.append([lm[0]-centre[0], lm[1]-centre[1], lm[2]-centre[2]])
        
        #Calculate joint angles for each finger joint
        thumbA = compute_joint_angle(result,1,2,3,4)
        indexA = compute_joint_angle(result,5,6,7,8)
        middleA = compute_joint_angle(result,9,10,11,12)
        ringA = compute_joint_angle(result,13,14,15,16)
        littleA = compute_joint_angle(result,17,18,19,20)
        
        x = []
        x.extend(thumbA)
        x.extend(indexA)
        x.extend(middleA)
        x.extend(ringA)
        x.extend(littleA)
        print(x)

[31.758455452602334, 9.822205681635875, 9.43943614032475, 15.621611214617552, 10.635797231643703, 10.311500278416455, 55.038300782607074, 35.76675909093376, 16.374808110989164, 55.39464834362752, 32.479910081517126, 22.31793315974337, 49.80877343015686, 16.046344855772, 16.94303844954971]
[27.880991280894925, 24.257732848286675, 11.57295248177154, 48.752030267299794, 5.678517384613107, 13.222548299723858, 41.612604321085456, 26.61359609324489, 20.85994387977768, 37.82183504080737, 105.0348070963538, 61.61026265420436, 21.600939514206402, 35.100468663244506, 14.941313709398372]
[21.586064370320106, 11.395455954443237, 15.657069326057531, 43.70226959681779, 9.332006945669498, 2.3627076811558796, 47.426521015490856, 11.03351217972501, 1.7226652462363292, 32.95523528796078, 13.074858474835661, 1.5313210320974662, 38.8572251389111, 22.856427895936026, 3.827834228899766]
[46.17727327033811, 15.512289244598785, 26.835123413851587, 44.16803800395692, 26.25841459606283, 38.03851655780797, 59.27

[42.804645207604324, 25.53549465414874, 8.209381823231238, 56.05309481420946, 9.675420201111448, 12.905857923740632, 42.9731828732905, 10.609086771478246, 10.285133754671554, 38.16599510084733, 14.045354500949003, 25.051705107011802, 35.49487475031066, 6.332669644519313, 15.319925400601324]
[36.226583847337395, 9.489431679145175, 4.173242268399144, 33.02765611343327, 53.167230181544916, 20.513493001613917, 51.34549877927534, 41.24068335457725, 16.18926182124084, 50.373937353814824, 24.76671251495747, 14.362771486467686, 30.0146310192989, 6.026730632815788, 9.603792809448162]
[40.96620400185099, 30.770623091300354, 9.28037914342183, 24.952967676997655, 7.015108070835517, 10.315719997501454, 40.8039404132285, 24.239416747860094, 6.35899265045302, 45.32998958057918, 30.601546235222628, 11.196133855382683, 53.59101249449289, 32.74640854417092, 16.50201481049501]
[46.116984410177174, 27.400090678078254, 3.5484366701055734, 23.143824524963556, 6.072352660752402, 11.664286720613594, 39.726410

[50.9395971788711, 26.183344444824804, 6.994256718634332, 68.19143871749776, 23.78388367743469, 22.165348512075134, 90.2069458017083, 41.728646595007326, 10.836784575208082, 92.33077953463075, 38.80989939834295, 12.906101386083854, 78.83535739546441, 40.81300145106202, 28.516276978278846]
[37.14380726181408, 2.4006685454303875, 23.19846310521765, 45.68169335912337, 88.29041048303665, 27.62893029016283, 52.792075718970544, 83.79379883311167, 32.81196419311508, 53.16486677223457, 92.06376998571744, 36.99067046343897, 34.19510318567487, 4.093497077163512, 10.808832768098906]
[44.02318592184933, 12.392837876931672, 25.69800139457485, 45.12674591790042, 9.856250886568896, 22.93331294142517, 69.63468819207833, 23.911863780658308, 13.769578822307075, 68.87363709624968, 26.617905475514807, 15.36041146881629, 20.74682299497865, 16.769399143642655, 6.314245829628658]
[32.31206685854526, 10.84960354216718, 5.975088946402934, 83.05768772579252, 17.271377835159527, 5.954278956420353, 63.54540865924

[35.7067245387927, 4.75018523057721, 23.021232088310416, 73.64702607540148, 24.11835233338461, 9.604212178507971, 61.651173879852934, 28.39175996855157, 7.4822469441150385, 51.3748589891263, 33.72245495741075, 15.566998037770736, 17.57627126210324, 4.44484226540452, 9.531164772102995]
[6.856030615801481, 7.485305224878941, 14.704734045412229, 29.61789845678831, 46.47773116184299, 12.463769139869019, 13.30695350799492, 114.60888641477312, 13.57686880284922, 5.848147098920966, 86.30201255165198, 59.057706851559566, 29.52715905078881, 6.402376680539688, 13.901133042546322]
[8.153645908608452, 14.902191667788193, 3.375910622761571, 69.41625695585589, 47.854212680883094, 21.06710675517636, 60.25433247865143, 26.41803278944394, 12.143416098744433, 36.21110269709151, 12.652951790959012, 11.645766378685764, 19.604758939268248, 9.856608577504993, 0.9316797492619232]
[34.126962330046105, 20.50325597056746, 9.895078585673172, 19.04400795304703, 2.840192107335507, 14.229260552761543, 49.1822254184

[44.13485188895956, 20.45098454088061, 10.193083916605575, 56.58184309989988, 23.290523294484327, 25.309344531877453, 52.27701973372722, 40.28925678041497, 75.63623093493894, 37.46810844717, 28.66121792039429, 36.24440127647759, 35.08078803179133, 16.215211285395924, 12.623294260065428]
[26.625687966593176, 8.41703764659811, 8.759021172268378, 65.18855198067598, 7.185665823348551, 7.17836795355333, 50.219439987169416, 7.743765430114756, 3.4932053941505217, 38.54977175407921, 4.981299107321792, 2.3617164442887058, 25.111367408153473, 4.096412514502847, 9.690747641699721]
[50.85505877459199, 23.172453688547453, 16.88417108365978, 40.23081754610321, 12.70495982412289, 8.47619908033535, 44.12288328373187, 13.503177901548472, 3.6126644188831003, 39.262144978475035, 58.92368158878255, 21.827647044393245, 28.626277583788866, 29.196952099689614, 8.945348894230044]
[48.28153113188734, 23.229654178343953, 14.402289815006673, 49.49760177788786, 11.175956296100093, 15.598645781561643, 52.162784961

...

[36.77301473437784, 52.85261361562013, 19.922807496859072, 12.277783331436245, 30.753981162018363, 19.467182265173737, 8.653969356009394, 33.82137030536092, 19.224061818387934, 11.07096693968139, 31.515389684886934, 22.567250676791136, 16.082367899124307, 20.506747638409514, 18.503263583450938]
[42.16165284735322, 17.803884151562105, 31.78055011850707, 10.452707231213488, 11.000750310596533, 10.648925760162278, 18.73725265848762, 10.612938774953971, 13.176041492442774, 21.362503853355843, 8.040356845690237, 16.04222345000278, 22.215078909531506, 10.76580300625444, 15.001588997280397]
[47.396425629514184, 47.512020944250985, 25.107209246841336, 45.55592115151693, 25.367856272947208, 3.291407256828161, 39.671461144847015, 29.342740998553058, 5.761632219314706, 36.45075873582848, 25.73647926163024, 9.477213737672981, 42.8390629943925, 11.457407090307091, 6.379218354990869]
[32.666180174668874, 13.84542416699853, 27.70402748027837, 32.89625132023465, 30.177361362862527, 6.913390670146498, 

[53.01207924482978, 17.098337345912473, 8.749577910216752, 68.46737722048358, 52.844857064413105, 38.6363681977868, 75.0051129473612, 75.98424699039685, 39.844135136619755, 81.08028874236338, 78.67901356422246, 47.04978890967791, 94.43460731587459, 65.97727776550158, 38.97847033560402]
[26.72758130437927, 26.057070737119666, 39.72444971688436, 31.007436606876187, 29.46004041551365, 9.47863036473497, 22.242566261928307, 30.981515905629575, 4.596080909971408, 22.48270910705552, 20.633605901781667, 6.314252871931979, 28.302695829930716, 11.543768054086769, 8.680128992699972]
[37.15472230857771, 27.871013782166113, 36.17602570839998, 21.00204560353531, 16.062360535319264, 10.357397989278695, 8.068456100085461, 20.530494910539456, 5.199148140375912, 14.49068122101074, 18.532840613126762, 5.119316729837389, 30.84802579221724, 8.0862536513389, 4.6064864276288375]
[27.39321950338762, 55.098092404392894, 53.447806996365806, 29.091858987559895, 30.58832630959891, 20.68873011299728, 16.5587043341

[47.97318234980004, 18.415039587744253, 2.802610986773474, 26.096081868807925, 23.359483143210785, 16.84759707553938, 19.487466453028336, 31.238192670621448, 17.288161522252732, 23.777351944651453, 30.579625057609896, 12.52346020418456, 35.011277226744376, 23.066581455716065, 11.58514419595654]
[36.317146643984735, 29.386443129685354, 21.310056083642092, 15.948373282353074, 21.507693664541485, 13.055782152176203, 4.881912151392955, 22.70835211816176, 8.385786391939918, 10.776612563129197, 18.787838227068832, 10.412071796900396, 27.422078688815947, 12.616450594641329, 6.220081002475691]
[21.767625029285416, 38.481924778898495, 54.19348549179705, 21.50246781982888, 17.910716463450473, 7.849263272282345, 12.995024132924863, 17.595473021577135, 7.480385981096493, 14.301992455938922, 6.89301029215483, 6.352589271843648, 25.740118992829732, 6.926258555927809, 5.862740304325935]
[23.87240865462484, 31.998833152415777, 66.83682168583898, 26.24955117755725, 15.58900359319314, 7.051245854710338,

[35.88784024599612, 20.513904762576534, 5.502783634606599, 5.815232860213941, 19.358511983629313, 13.285544171255403, 2.0679526184888792, 24.430124272315606, 18.772645985024916, 6.563167197359277, 23.04313007915456, 12.791821381852785, 11.44193332949485, 18.703011123320696, 15.202118392042978]
[30.356076273242802, 22.65209290930632, 52.569340597258, 17.552120221260385, 14.373085665215811, 7.1970864819381095, 8.425315396744915, 18.83972064699344, 1.6298500762706956, 9.283099682842034, 14.513674978532773, 2.146488590293589, 18.356753674185537, 6.494722327280368, 5.1066073853659795]
[27.53803960502023, 16.30405124651543, 21.08460745441016, 24.300925926552278, 17.27925938527424, 10.932602577902795, 12.69563317222119, 29.20662218741276, 5.317976165813513, 13.364161309505644, 30.10104736364642, 5.3878866161664165, 22.574123869812546, 21.83217724581707, 5.445484469844648]
[27.528044192643637, 24.191751655357734, 12.869503882504766, 20.87309028937646, 8.678930677010399, 7.398341097782756, 11.3

[46.82237162093781, 20.22032050957928, 7.029517922704855, 39.98579592052665, 24.92159240524495, 17.821867566592882, 38.20002786098759, 33.41509603714621, 15.338149674000327, 35.42067845560327, 35.50259835337363, 15.938705361993993, 36.04462421376177, 29.863794192016474, 14.99899560680155]
[61.60046908866401, 25.538494735442228, 10.96835217281899, 22.076444579087717, 25.495309697433047, 8.430054709662912, 15.497013463052992, 35.25966591664596, 13.460465771663351, 8.833994144100606, 39.77447336941502, 10.493296084956542, 9.760467360725901, 31.00466452517629, 15.519018431786657]
[33.21087690246036, 10.327787814128477, 25.40452264422336, 24.345084571266714, 21.085009864363435, 8.331203907440287, 17.770025767690978, 18.489669850665763, 4.2440418649574, 14.91592806918022, 13.600392802413744, 4.215180688360053, 20.460775373756785, 5.199863395750703, 3.348069642444904]
[47.023501178688605, 17.978610624045096, 4.84436500671148, 39.023716019376664, 27.286534543882286, 13.835253635705548, 28.6919